# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet
#tfdet.core.util.set_seed(777) #set seed
device = tfdet.core.util.select_device(0) #set device

# 1. Init Dataset

In [3]:
import os
path = "./balloon"
if not os.path.exists(path):
    !wget -qq --no-check-certificate http://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
    !unzip -qq balloon_dataset.zip
print(tfdet.dataset.util.tree_dir(path))

balloon/
    train/
        605521662_a470fef77f_b.jpg
        9330497995_4cf0438cb6_k.jpg
        5178670692_63a4365c9c_b.jpg
        ...
    val/
        4838031651_3e7b5ea5c7_b.jpg
        16335852991_f55de7958d_k.jpg
        24631331976_defa3bb61f_k.jpg
        ...


In [4]:
train_shape = [512, 512]
image_shape = [320, 320]

label = tfdet.dataset.balloon.LABEL #["background", "balloon"]
n_class = len(label)

tr_path = os.path.join(path, "train/via_region_data.json")
te_path = os.path.join(path, "val/via_region_data.json")

for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

tr_pipe = tfdet.dataset.balloon.load_pipe(tr_path, mask = False)
tr_pipe = tfdet.dataset.pipeline.load(tr_pipe)
tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe) #for train_model
if True: #If the number of training data is small, it is better to apply weak augmentation without spatial transform, or offline augmentation.
    tr_pipe = tfdet.dataset.pipeline.weak_augmentation(tr_pipe, crop_shape = None) #If crop_shape is shape or ratio, apply random_crop.
    tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = train_shape, keep_ratio = True)
elif True:
    #tr_pipe = tfdet.dataset.pipeline.albumentations(tr_pipe)
    #tr_pipe = tfdet.dataset.pipeline.mmdet_augmentation(tr_pipe, image_shape = [int(min(train_shape) * 0.1), int(max(train_shape) * 2)], crop_shape = train_shape, resize_mode = "range", keep_ratio = True, shape_divisor = 32)
    tr_pipe = tfdet.dataset.pipeline.mmdet_augmentation(tr_pipe, image_shape = train_shape, crop_shape = None, keep_ratio = True, shape_divisor = 32)
else:
    tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = train_shape, keep_ratio = True)
    #tr_pipe = tfdet.dataset.pipeline.yolo_augmentation(tr_pipe, sample_x_true = tr_pipe.cache("./sample_cache"), image_shape = train_shape, p_mix_up = 0.15, p_copy_paste = 0.)
    tr_pipe = tfdet.dataset.pipeline.yolo_augmentation(tr_pipe, sample_x_true = tr_pipe, sample_cache = "./sample_cache", image_shape = train_shape, p_mix_up = 0.15, p_copy_paste = 0.)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, image_shape = train_shape, mode = "both")
tr_pipe = tfdet.dataset.pipeline.filter_annotation(tr_pipe, min_scale = 2, min_instance_area = 1)
tr_pipe = tfdet.dataset.pipeline.collect(tr_pipe, keys = ["x_true", "y_true", "bbox_true"])
tr_pipe = tfdet.dataset.pipeline.label_encode(tr_pipe, label = tfdet.dataset.balloon.LABEL)
tr_pipe = tfdet.dataset.pipeline.normalize(tr_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375], bbox_normalize = True)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, max_pad_size = 200)
tr_pipe = tfdet.dataset.pipeline.cast(tr_pipe)
#tr_pipe = tfdet.dataset.pipeline.reshape(tr_pipe, map = {"x_true":[*train_shape, 3], "y_true":[200, 1], "bbox_true":[200, 4], "mask_true":[200, *train_shape, 1]})

te_pipe = tfdet.dataset.balloon.load_pipe(te_path, mask = False)
te_pipe = tfdet.dataset.pipeline.load(te_pipe)
te_pipe = tfdet.dataset.pipeline.args2dict(te_pipe) #for train_model
te_pipe = tfdet.dataset.pipeline.resize(te_pipe, image_shape = image_shape, keep_ratio = True)
te_pipe = tfdet.dataset.pipeline.pad(te_pipe, image_shape = image_shape, mode = "both")
te_pipe = tfdet.dataset.pipeline.filter_annotation(te_pipe, min_scale = 2, min_instance_area = 1)
te_pipe = tfdet.dataset.pipeline.collect(te_pipe, keys = ["x_true", "y_true", "bbox_true"])
te_pipe = tfdet.dataset.pipeline.label_encode(te_pipe, label = tfdet.dataset.balloon.LABEL)
te_pipe = tfdet.dataset.pipeline.normalize(te_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375], bbox_normalize = True)
te_pipe = tfdet.dataset.pipeline.pad(te_pipe, max_pad_size = 100)
te_pipe = tfdet.dataset.pipeline.cast(te_pipe)
#te_pipe = tfdet.dataset.pipeline.reshape(te_pipe, map = {"x_true":[*image_shape, 3], "y_true":[100, 1], "bbox_true":[100, 4], "mask_true":[100, *image_shape, 1]})

# 2. Build Detector

In [5]:
with device:
    x = tf.keras.layers.Input(shape = [None, None, 3], name = "x_true")
    out = tfdet.model.detector.effdet_lite_d0(x, n_class = n_class, scale = [24, 48, 96, 192, 320], ratio = [0.5, 1, 2], octave = 3)
    model = tfdet.model.train.effdet.train_model(x, *out, assign = tfdet.core.assign.max_iou,
                                                 focal = True, regularize = True, weight_decay = 1e-4,
                                                 proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.25)

# 3. Train

3-0. Init HyperParameter

In [6]:
import os, shutil

epoch = 300
#train_batch_size = 16
train_batch_size = 4
batch_size = 16
save_path = "./learn/epoch@{epoch:03d}-metric@{mean_average_precision:.4f}-loss@{loss:.4f}-val_loss@{val_loss:.4f}.h5"

learning_rate = 1e-2
dst_learning_rate = 1e-4
momentum = 0.9
nesterov = True
warm_up_epoch = 5

if os.path.exists(os.path.dirname(save_path)):
    shutil.rmtree(os.path.dirname(save_path))
os.makedirs(os.path.dirname(save_path), exist_ok = True)

train_pipe = tfdet.dataset.pipeline.key_map(tr_pipe, map = {"x_true":x.name}, batch_size = train_batch_size, shuffle = True, prefetch = True)
test_pipe = tfdet.dataset.pipeline.key_map(te_pipe, map = {"x_true":x.name}, batch_size = batch_size, prefetch = True)

metric = tfdet.callback.CoCoMeanAveragePrecision(test_pipe, label = label, name = "mean_average_precision")
save = tf.keras.callbacks.ModelCheckpoint(save_path, monitor = "mean_average_precision", mode = "max", save_best_only = True, save_weights_only = True, save_freq = "epoch", verbose = 0)
#scheduler = tfdet.callback.WarmUpCosineLearningRateScheduler(cycle = epoch, decay_rate = dst_learning_rate / learning_rate, warm_up_epoch = warm_up_epoch)
scheduler = tfdet.callback.WarmUpCosineLearningRateSchedulerStep(cycle = epoch, decay_rate = dst_learning_rate / learning_rate, total_step = (train_pipe.cardinality() if 0 < train_pipe.cardinality() else None), warm_up_epoch = warm_up_epoch) #total step None > dynamic total step
logger = tf.keras.callbacks.CSVLogger(os.path.join(os.path.dirname(save_path), "logger.csv"), separator = ",")

3-1. Default

In [7]:
with device:
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
    model.compile(optimizer = optimizer)

    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Epoch 1/50

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |1300    |0.034 |0.0001           |0.0005              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |1300    |0.034 |0.0001           |0.0005              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
128/128 - 109s - loss: 99.6350 - score_accuracy: 0.3967 - score_loss: 99.0866 - regress_loss: 0.5482 - val_loss: 10.7235 - val_score_accuracy: 0.7609 - val_score_loss: 10.0707 - val_regress_loss: 

3-2. Stochastic Weight Averaging(SWA)

In [7]:
import traceback

with device:
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
    try:
        !pip install tensorflow_addons
        import tensorflow_addons as tfa
        #optimizer = tfa.optimizers.SGDW(learning_rate = learning_rate, weight_decay = 1e-4, momentum = momentum, nesterov = nesterov)
        optimizer = tfa.optimizers.SWA(optimizer, start_averaging = epoch // 3, average_period = 10)
    except:
        print(traceback.format_exc())
    model.compile(optimizer = optimizer)

    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)0
    if name in ["sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.5 MB/s 
Epoch 1/50

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |1300    |0.004 |0.0              |0.0001              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |1300    |0.004 |0.0              |0.0001              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
128/128 - 130s - loss: 94.4999 - score_a

3-2. Exponential Moving Average(EMA)

In [7]:
with device:
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
    model.compile(optimizer = optimizer)

    ema = tfdet.callback.EMA(step = 64 // train_batch_size, auto_apply = True, warm_up_epoch = warm_up_epoch)

    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [ema, metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Epoch 1/50

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |1300    |0.014 |0.0              |0.0001              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |1300    |0.014 |0.0              |0.0001              |0.0                  |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
128/128 - 134s - loss: 101.5919 - score_accuracy: 0.5235 - score_loss: 101.0272 - regress_loss: 0.5643 - val_loss: 13.4430 - val_score_accuracy: 0.4332 - val_score_loss: 12.7908 - val_regress_loss

# 4. Evaluate

In [8]:
model_path = tfdet.dataset.util.list_dir(os.path.dirname(save_path), "h5")
model_path = sorted(model_path, key = lambda x: x.split("metric@")[1].split("-")[0], reverse = True)[0]
model.load_weights(model_path)

with device:
    nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.25)(out)
    model = tf.keras.Model(x, nms_out)

metric = tfdet.core.metric.CoCoMeanAveragePrecision(label = label)
for te_dict in test_pipe:
    x_true, y_true, bbox_true = te_dict["x_true"], te_dict["y_true"], te_dict["bbox_true"]
    y_pred, bbox_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, bbox_true, y_pred, bbox_pred)
print(metric.summary_text)

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |36      |0.358 |0.3333           |0.4886              |0.404                |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |36      |0.358 |0.3333           |0.4886              |0.404                |
+----------+--------+--------+------+-----------------+--------------------+---------------------+


# 5. Load

In [9]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    out = tfdet.model.detector.effdet_lite_d0(x, n_class = n_class, scale = [24, 48, 96, 192, 320], ratio = [0.5, 1, 2], octave = 3)
    nms_out = tfdet.model.postprocess.effdet.FilterDetection(proposal_count = 100, iou_threshold = 0.5, score_threshold = 0.25)(out)
    model = tf.keras.Model(x, nms_out)
    model.load_weights(model_path)
    
metric = tfdet.core.metric.CoCoMeanAveragePrecision(label = label)
for te_dict in test_pipe:
    x_true, y_true, bbox_true = te_dict["x_true"], te_dict["y_true"], te_dict["bbox_true"]
    y_pred, bbox_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, bbox_true, y_pred, bbox_pred)
print(metric.summary_text)

+----------+--------+--------+------+-----------------+--------------------+---------------------+
|label     |num_true|num_pred|recall|average_precision|average_precision@.5|average_precision@.75|
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|background|0       |0       |0.0   |0.0              |0.0                 |0.0                  |
|balloon   |50      |36      |0.358 |0.3333           |0.4886              |0.404                |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
|summary   |50      |36      |0.358 |0.3333           |0.4886              |0.404                |
+----------+--------+--------+------+-----------------+--------------------+---------------------+
